In [1]:
!pip install pandasql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26781 sha256=762366060ef493b41db9434eb58738d757783028cfc5dccd9b574770fd2c6935
  Stored in directory: /root/.cache/pip/wheels/5c/4b/ec/41f4e116c8053c3654e2c2a47c62b4fca34cc67ef7b55deb7f
Successfully built pandasql


In [2]:
import pandas as pd
import numpy as np
import pandasql as ps
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV, KFold, train_test_split, StratifiedKFold

# Loading Datasets

In [5]:
df = pd.read_csv("../data/full_dataset.csv")

In [7]:
reddit = pd.read_csv('../data/datafiles/Reddit_Sentiments_equalweight.csv')

In [8]:
twitter = pd.read_csv('../data/datafiles/Twitter_Sentiments_Average.csv')

In [9]:
df

,CoefficientVariationofStockPrice,PercentageChangefromprevreportedEPS,PriceChange,QuestionsSentiment,ConferenceSentiment,Has_QnA,Reddit_Sentiment,RedditSD,TwitterSentiment,TwitterSD,...,Energy,Financial Services,Healthcare,Industrials,Real Estate,Technology,quarter_1,quarter_2,quarter_3,quarter_4
0,0.106351,-0.269231,-0.122655,-0.762857,0.666667,0,0.098144,0.408199,0.103101,0.489937,...,0,0,0,0,0,0,0.0,0.0,1.0,0.0
1,0.219666,-0.217391,0.579668,-0.762857,0.555556,0,0.018184,0.387645,0.126711,0.480920,...,0,0,0,0,0,0,0.0,1.0,0.0,0.0
2,0.180643,-1.600000,-0.535234,-0.762857,0.200000,0,0.007145,0.365686,0.087753,0.491585,...,0,0,0,0,0,0,1.0,0.0,0.0,0.0
3,0.086445,-0.315789,-0.160201,-0.762857,0.454545,0,0.249586,0.360750,0.124391,0.499839,...,0,0,0,0,0,0,0.0,0.0,0.0,1.0
4,0.164528,0.545455,-0.324167,-0.762857,0.384418,0,0.155589,0.377683,0.148988,0.489504,...,0,0,0,0,0,0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,0.025595,-0.120482,0.014390,-0.454545,-0.333333,1,0.028897,0.352831,0.213262,0.419719,...,0,0,0,0,0,0,1.0,0.0,0.0,0.0
975,0.016441,0.128788,0.023988,-0.166667,-0.333333,1,0.046342,0.258866,0.247028,0.414870,...,0,0,0,0,0,0,0.0,0.0,0.0,1.0
976,0.012858,0.168224,-0.005354,-0.400000,-0.333333,1,0.117191,0.320746,0.110092,0.423049,...,0,0,0,0,0,0,0.0,0.0,1.0,0.0
977,0.081405,-0.289655,-0.074019,-0.333333,-0.333333,1,0.061396,0.416460,0.135574,0.415822,...,0,0,0,0,0,0,0.0,1.0,0.0,0.0


In [10]:
df['Reddit_Sentiment'] = reddit['Reddit_Sentiment']

In [11]:
df['TwitterSentiment'] = twitter['Twitter Sentiment']

In [12]:
df[df.isna().any(axis = 1)]

,CoefficientVariationofStockPrice,PercentageChangefromprevreportedEPS,PriceChange,QuestionsSentiment,ConferenceSentiment,Has_QnA,Reddit_Sentiment,RedditSD,TwitterSentiment,TwitterSD,...,Energy,Financial Services,Healthcare,Industrials,Real Estate,Technology,quarter_1,quarter_2,quarter_3,quarter_4
298,0.104221,-0.469697,0.102156,-0.250000,0.480000,1,NaN,0.000000,0.104072,0.467092,...,0,0,1,0,0,0,0.0,0.0,1.0,0.0
299,0.098578,-0.538530,0.046784,-0.714286,0.666667,1,NaN,0.000000,0.197838,0.500382,...,0,0,1,0,0,0,0.0,1.0,0.0,0.0
300,0.251198,0.285714,-0.580915,-0.666667,0.696203,1,NaN,0.000000,0.151969,0.490923,...,0,0,1,0,0,0,1.0,0.0,0.0,0.0
301,0.120399,0.400929,-0.168023,-0.555556,0.696203,1,NaN,0.000000,0.230078,0.466391,...,0,0,1,0,0,0,0.0,0.0,0.0,1.0
302,0.274975,1.098592,1.373557,-0.714286,0.620253,1,NaN,0.000000,0.226563,0.438698,...,0,0,1,0,0,0,0.0,0.0,1.0,0.0
303,0.122977,-4.463768,0.198909,-0.384615,0.545455,1,NaN,0.000000,0.196580,0.422407,...,0,0,1,0,0,0,0.0,1.0,0.0,0.0
304,0.268340,-0.406780,1.284302,-0.454545,0.449275,1,NaN,0.000000,0.194724,0.427648,...,0,0,1,0,0,0,1.0,0.0,0.0,0.0
305,0.069085,0.387097,-0.099286,-1.000000,0.389831,1,NaN,0.000000,0.148994,0.415968,...,0,0,1,0,0,0,0.0,0.0,0.0,1.0
306,0.136671,0.000000,0.593054,-0.833333,0.444444,1,NaN,0.000000,0.201550,0.444845,...,0,0,1,0,0,0,0.0,0.0,1.0,0.0
307,0.276831,-0.054054,1.076599,-0.400000,0.666667,1,NaN,0.000000,0.203575,0.436939,...,0,0,1,0,0,0,0.0,1.0,0.0,0.0


In [13]:
df['Reddit_Sentiment'] = df['Reddit_Sentiment'].fillna(df['Reddit_Sentiment'].median())

In [14]:
df['TwitterSentiment'] = df['TwitterSentiment'].fillna(df['TwitterSentiment'].median())

In [15]:
df[df.isna().any(axis = 1)]

,CoefficientVariationofStockPrice,PercentageChangefromprevreportedEPS,PriceChange,QuestionsSentiment,ConferenceSentiment,Has_QnA,Reddit_Sentiment,RedditSD,TwitterSentiment,TwitterSD,...,Energy,Financial Services,Healthcare,Industrials,Real Estate,Technology,quarter_1,quarter_2,quarter_3,quarter_4


# Train-Test Split

In [16]:
all_features = df.columns.tolist()

In [17]:
all_features

['CoefficientVariationofStockPrice',
 'PercentageChangefromprevreportedEPS',
 'PriceChange',
 'QuestionsSentiment',
 'ConferenceSentiment',
 'Has_QnA',
 'Reddit_Sentiment',
 'RedditSD',
 'TwitterSentiment',
 'TwitterSD',
 'SocialMediaCovariance',
 'TwitterSkew',
 'TwitterKurtosis',
 'RedditSkew',
 'RedditKurtosis',
 'EPSEstimate',
 'Has_Reddit',
 'Communication Services',
 'Consumer Cyclical',
 'Consumer Defensive',
 'Energy',
 'Financial Services',
 'Healthcare',
 'Industrials',
 'Real Estate',
 'Technology',
 'quarter_1',
 'quarter_2',
 'quarter_3',
 'quarter_4']

In [18]:
X = df

In [19]:
y = pd.read_csv("../data/full_labels.csv")

In [20]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

In [21]:
X_train.shape, X_test.shape

((783, 30), (196, 30))

In [22]:
y_train.shape, y_test.shape

((783, 1), (196, 1))

In [23]:
X_train

,CoefficientVariationofStockPrice,PercentageChangefromprevreportedEPS,PriceChange,QuestionsSentiment,ConferenceSentiment,Has_QnA,Reddit_Sentiment,RedditSD,TwitterSentiment,TwitterSD,...,Energy,Financial Services,Healthcare,Industrials,Real Estate,Technology,quarter_1,quarter_2,quarter_3,quarter_4
378,0.307267,-0.557576,1.409556,-0.714286,0.578947,1,0.088727,0.423146,0.235674,0.441965,...,0,0,0,0,0,0,1.0,0.0,0.0,0.0
468,0.242076,0.306452,1.378303,-0.666667,0.466667,1,0.061742,0.433917,0.285148,0.451483,...,1,0,0,0,0,0,1.0,0.0,0.0,0.0
409,0.043960,0.057471,-0.009751,-0.750000,0.652174,1,0.000000,0.000000,0.291313,0.440203,...,0,0,0,0,0,0,1.0,0.0,0.0,0.0
445,0.106001,0.000000,-0.025346,-0.714286,0.566667,1,0.083508,0.441184,0.181327,0.412363,...,0,0,0,0,0,1,0.0,0.0,0.0,1.0
207,0.036320,-0.256881,-0.100652,-0.272727,0.761905,1,0.159395,0.318592,0.175525,0.434037,...,0,0,0,0,0,1,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
669,0.078014,-0.906250,0.114644,-0.333333,0.555556,1,0.124989,0.420355,0.134098,0.309138,...,0,0,0,0,0,0,0.0,1.0,0.0,0.0
576,0.105664,0.105263,-0.402886,-0.272727,0.696970,1,0.015133,0.422794,0.117354,0.485417,...,0,0,0,0,0,1,0.0,0.0,1.0,0.0
606,0.218293,-0.257143,0.484055,0.200000,0.673469,1,0.189716,0.405775,0.166617,0.399143,...,0,0,0,0,0,1,1.0,0.0,0.0,0.0
594,0.096284,-0.536232,0.192951,-0.500000,0.777778,1,0.000000,0.177206,0.213787,0.402587,...,0,0,0,0,0,0,1.0,0.0,0.0,0.0


In [ ]:
X_train.to_csv('../data/alternatives_alpha_datasets/train_ew.csv', index = False)
X_test.to_csv('../data/alternatives_alpha_datasets/test_ew.csv', index = False)
